# Experiment 006: Simulated Annealing from Scratch

Implement SA to improve from Zaburo's 87.99 toward the target 68.89.

Focus on small N values (2-10) first since they have the highest per-tree score contribution.

In [1]:
import math
import os
import copy
import random
from decimal import Decimal, getcontext
import numpy as np
import pandas as pd
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import json
from tqdm import tqdm

# Use high precision
getcontext().prec = 25
scale_factor = Decimal('1e15')

random.seed(42)
np.random.seed(42)

print(f"Decimal precision: {getcontext().prec}")

Decimal precision: 25


In [2]:
# ChristmasTree class
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        self.polygon = self._create_polygon()
    
    def _create_polygon(self):
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (float(Decimal('0.0') * scale_factor), float(tip_y * scale_factor)),
            (float(top_w / Decimal('2') * scale_factor), float(tier_1_y * scale_factor)),
            (float(top_w / Decimal('4') * scale_factor), float(tier_1_y * scale_factor)),
            (float(mid_w / Decimal('2') * scale_factor), float(tier_2_y * scale_factor)),
            (float(mid_w / Decimal('4') * scale_factor), float(tier_2_y * scale_factor)),
            (float(base_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(base_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(mid_w / Decimal('4')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(mid_w / Decimal('2')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(top_w / Decimal('4')) * scale_factor), float(tier_1_y * scale_factor)),
            (float(-(top_w / Decimal('2')) * scale_factor), float(tier_1_y * scale_factor)),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        return affinity.translate(rotated,
                                  xoff=float(self.center_x * scale_factor),
                                  yoff=float(self.center_y * scale_factor))
    
    def update_polygon(self):
        self.polygon = self._create_polygon()
    
    def clone(self):
        return ChristmasTree(str(self.center_x), str(self.center_y), str(self.angle))

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
def check_overlap(trees):
    """Check for overlaps."""
    polygons = [t.polygon for t in trees]
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                return True
    return False

def calculate_side_length(trees):
    """Calculate bounding box side length."""
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    minx = Decimal(str(bounds[0])) / scale_factor
    miny = Decimal(str(bounds[1])) / scale_factor
    maxx = Decimal(str(bounds[2])) / scale_factor
    maxy = Decimal(str(bounds[3])) / scale_factor
    width = maxx - minx
    height = maxy - miny
    return float(max(width, height))

def calculate_score(trees, n):
    """Calculate score for N trees."""
    side = calculate_side_length(trees)
    return (side ** 2) / n

print("Helper functions defined")

Helper functions defined


In [4]:
# Load Zaburo's valid solution
def parse_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return val[1:]
    return str(val)

def load_submission(path):
    df = pd.read_csv(path)
    trees_by_n = {}
    for _, row in df.iterrows():
        id_parts = row['id'].split('_')
        n = int(id_parts[0])
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        if n not in trees_by_n:
            trees_by_n[n] = []
        trees_by_n[n].append(ChristmasTree(center_x=x, center_y=y, angle=deg))
    return trees_by_n

zaburo_path = '/home/code/experiments/005_zaburo_rowbased/submission.csv'
all_trees = load_submission(zaburo_path)
print(f"Loaded Zaburo solution with {len(all_trees)} N values")

# Calculate initial score
initial_score = sum(calculate_score(all_trees[n], n) for n in range(1, 201))
print(f"Initial score: {initial_score:.6f}")

Loaded Zaburo solution with 200 N values


Initial score: 87.991248


In [5]:
def simulated_annealing(trees, n, max_iters=5000, T0=1.0, T_min=0.00001, alpha=0.999):
    """
    Simulated annealing for a single N value.
    
    Moves:
    - translate: move a tree by small amount
    - rotate: rotate a tree by small angle
    """
    current_trees = [t.clone() for t in trees]
    current_score = calculate_score(current_trees, n)
    best_trees = [t.clone() for t in current_trees]
    best_score = current_score
    
    T = T0
    accepted = 0
    rejected_overlap = 0
    rejected_worse = 0
    
    for iteration in range(max_iters):
        # Choose a random tree and move type
        tree_idx = random.randint(0, len(current_trees) - 1)
        move_type = random.choice(['translate', 'rotate'])
        
        # Save old state
        old_x = current_trees[tree_idx].center_x
        old_y = current_trees[tree_idx].center_y
        old_angle = current_trees[tree_idx].angle
        
        # Make move
        if move_type == 'translate':
            # Adaptive step size based on temperature
            step = 0.1 * (T / T0) + 0.01
            dx = random.uniform(-step, step)
            dy = random.uniform(-step, step)
            current_trees[tree_idx].center_x += Decimal(str(dx))
            current_trees[tree_idx].center_y += Decimal(str(dy))
        else:  # rotate
            step = 10 * (T / T0) + 1
            dangle = random.uniform(-step, step)
            current_trees[tree_idx].angle += Decimal(str(dangle))
        
        current_trees[tree_idx].update_polygon()
        
        # Check for overlaps
        if check_overlap(current_trees):
            # Revert
            current_trees[tree_idx].center_x = old_x
            current_trees[tree_idx].center_y = old_y
            current_trees[tree_idx].angle = old_angle
            current_trees[tree_idx].update_polygon()
            rejected_overlap += 1
            continue
        
        # Calculate new score
        new_score = calculate_score(current_trees, n)
        delta = new_score - current_score
        
        # Accept or reject
        if delta < 0 or random.random() < math.exp(-delta / T):
            current_score = new_score
            accepted += 1
            
            if current_score < best_score:
                best_score = current_score
                best_trees = [t.clone() for t in current_trees]
        else:
            # Revert
            current_trees[tree_idx].center_x = old_x
            current_trees[tree_idx].center_y = old_y
            current_trees[tree_idx].angle = old_angle
            current_trees[tree_idx].update_polygon()
            rejected_worse += 1
        
        # Cool down
        T = max(T * alpha, T_min)
    
    return best_trees, best_score, accepted, rejected_overlap, rejected_worse

print("SA function defined")

SA function defined


In [6]:
# Run SA on small N values (2-10) first - these have highest impact
print("Running SA on small N values (2-10)...")
print("="*60)

for n in range(2, 11):
    old_score = calculate_score(all_trees[n], n)
    
    # Run SA with more iterations for small N
    new_trees, new_score, accepted, rej_overlap, rej_worse = simulated_annealing(
        all_trees[n], n, max_iters=10000, T0=1.0, T_min=0.00001, alpha=0.9995
    )
    
    improvement = old_score - new_score
    
    if new_score < old_score:
        all_trees[n] = new_trees
        print(f"N={n:3d}: {old_score:.6f} -> {new_score:.6f} (improved by {improvement:.6f})")
    else:
        print(f"N={n:3d}: {old_score:.6f} -> {new_score:.6f} (no improvement)")
    
    print(f"       Accepted: {accepted}, Rejected (overlap): {rej_overlap}, Rejected (worse): {rej_worse}")

Running SA on small N values (2-10)...


N=  2: 0.720000 -> 0.655506 (improved by 0.064494)
       Accepted: 8662, Rejected (overlap): 529, Rejected (worse): 809


N=  3: 0.653333 -> 0.578656 (improved by 0.074677)
       Accepted: 9343, Rejected (overlap): 184, Rejected (worse): 473


N=  4: 0.765625 -> 0.724729 (improved by 0.040896)
       Accepted: 9488, Rejected (overlap): 268, Rejected (worse): 244


N=  5: 0.800000 -> 0.800000 (no improvement)
       Accepted: 9410, Rejected (overlap): 381, Rejected (worse): 209


N=  6: 0.666667 -> 0.666667 (no improvement)
       Accepted: 9237, Rejected (overlap): 556, Rejected (worse): 207


N=  7: 0.630000 -> 0.630000 (no improvement)
       Accepted: 9534, Rejected (overlap): 279, Rejected (worse): 187


N=  8: 0.551250 -> 0.551250 (no improvement)
       Accepted: 9547, Rejected (overlap): 340, Rejected (worse): 113


N=  9: 0.537778 -> 0.537778 (no improvement)
       Accepted: 9452, Rejected (overlap): 446, Rejected (worse): 102


N= 10: 0.484000 -> 0.484000 (no improvement)
       Accepted: 9251, Rejected (overlap): 654, Rejected (worse): 95


In [7]:
# Run SA on medium N values (11-50)
print("\nRunning SA on medium N values (11-50)...")
print("="*60)

for n in range(11, 51):
    old_score = calculate_score(all_trees[n], n)
    
    new_trees, new_score, accepted, rej_overlap, rej_worse = simulated_annealing(
        all_trees[n], n, max_iters=5000, T0=0.5, T_min=0.00001, alpha=0.999
    )
    
    improvement = old_score - new_score
    
    if new_score < old_score:
        all_trees[n] = new_trees
        print(f"N={n:3d}: {old_score:.6f} -> {new_score:.6f} (improved by {improvement:.6f})")
    # Only print if improved to reduce output


Running SA on medium N values (11-50)...


In [ ]:
# Run SA on large N values (51-200) with fewer iterations
print("\nRunning SA on large N values (51-200)...")
print("="*60)

improved_count = 0
for n in range(51, 201):
    old_score = calculate_score(all_trees[n], n)
    
    new_trees, new_score, accepted, rej_overlap, rej_worse = simulated_annealing(
        all_trees[n], n, max_iters=2000, T0=0.3, T_min=0.00001, alpha=0.998
    )
    
    if new_score < old_score:
        all_trees[n] = new_trees
        improved_count += 1

print(f"Improved {improved_count} out of 150 large N values")

In [ ]:
# Validate ALL N values
print("\nValidating all N values...")
overlapping_n = []

for n in range(1, 201):
    if check_overlap(all_trees[n]):
        overlapping_n.append(n)

print(f"Total N values with overlaps: {len(overlapping_n)}")
if overlapping_n:
    print(f"Overlapping N values: {overlapping_n}")
else:
    print("✓ All N values validated - NO OVERLAPS!")

In [ ]:
# Calculate final score
final_score = sum(calculate_score(all_trees[n], n) for n in range(1, 201))
per_n_scores = {n: calculate_score(all_trees[n], n) for n in range(1, 201)}

print(f"\nFinal Results:")
print(f"="*60)
print(f"Initial score (Zaburo): {initial_score:.6f}")
print(f"Final score (SA):       {final_score:.6f}")
print(f"Improvement:            {initial_score - final_score:.6f}")
print(f"Target:                 68.887226")
print(f"Gap to target:          {final_score - 68.887226:.6f}")

print(f"\nTop 10 score contributors:")
sorted_scores = sorted(per_n_scores.items(), key=lambda x: x[1], reverse=True)
for n, score in sorted_scores[:10]:
    print(f"N={n:3d}: score={score:.6f}")

In [ ]:
# Create submission DataFrame
index = [f'{n:03d}_{t}' for n in range(1, 201) for t in range(n)]
tree_data = []

for n in range(1, 201):
    for tree in all_trees[n]:
        tree_data.append([tree.center_x, tree.center_y, tree.angle])

cols = ['x', 'y', 'deg']
submission = pd.DataFrame(index=index, columns=cols, data=tree_data).rename_axis('id')

for col in cols:
    submission[col] = submission[col].astype(float).round(decimals=6)
    
for col in submission.columns:
    submission[col] = 's' + submission[col].astype('string')

print(f"Submission shape: {submission.shape}")

In [ ]:
# Save submission
os.makedirs('/home/submission', exist_ok=True)
submission.to_csv('/home/submission/submission.csv')
submission.to_csv('/home/code/experiments/006_sa_from_scratch/submission.csv')

# Save metrics
metrics = {
    'cv_score': final_score,
    'initial_score': initial_score,
    'improvement': initial_score - final_score,
    'overlapping_n_count': len(overlapping_n),
    'per_n_scores': {str(k): v for k, v in per_n_scores.items()}
}

with open('/home/code/experiments/006_sa_from_scratch/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"\nSubmission saved!")
print(f"CV Score: {final_score:.6f}")
print(f"Improvement from Zaburo: {initial_score - final_score:.6f}")